In [1]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
#from linearmodels.iv import IV2SLS
import functions as fun
import statsmodels.api as sm
import seaborn as sns

In [2]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
data = df[df['Market share'] != 0]

data.head(20)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [3]:
# Copy the dataframe
df2 = data.copy()

In [4]:
#Scale for better intepretation

df2['Price'] = df2['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
df2['HP'] = df2['HP']/10           #(Change in ms(%) for change in HP in 10)
df2['Range'] = df2['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [5]:
# Creating dummy for china
df2['China'] = (df2['Country'] == 'CN').astype(int)

In [6]:
# Take the log of the market share
df2['log_market_share'] = np.log(df2['Market share'])

# OLS

In [7]:
y = df2['log_market_share']
x = df2[['const', 'Price','Range', 'HP', 'Chargetime']]
dummies = df2[['China']]
X = pd.concat([x, dummies], axis=1)
print(X.shape)

(334, 6)


In [8]:
OLS_model = sm.OLS(y, X)
OLS_result = OLS_model.fit(cov_type='HC3')
# Print summary of the regression results
print(OLS_result.summary())

                            OLS Regression Results                            
Dep. Variable:       log_market_share   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.195
Method:                 Least Squares   F-statistic:                     21.01
Date:                Tue, 07 May 2024   Prob (F-statistic):           3.18e-18
Time:                        18:31:17   Log-Likelihood:                -716.37
No. Observations:                 334   AIC:                             1445.
Df Residuals:                     328   BIC:                             1468.
Df Model:                           5                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.4240      0.700     -9.171      0.0

In [9]:
y = df2['log_market_share']
x = df2[['const', 'Price', 'Range', 'HP', 'Chargetime']]
dummies = df2[[ 'China']]
X1 = pd.concat([x, dummies], axis=1)
print(df2.head(5))

    ID  Year  Market share Manufacturer   Model  Range      Price    HP  \
8    1  2021      0.010373       Aiways      U5   40.0  28.462170  20.1   
9    1  2022      0.005976       Aiways      U5   40.0  31.368183  20.1   
10   1  2023      0.002860       Aiways      U5   40.0  26.452400  20.1   
21   2  2023      0.000048       Aiways      U6   40.5  36.063800  21.4   
28   3  2019      0.040630         Audi  e-tron   37.5  97.970447  40.2   

    Chargetime Type Segment Country  Sales  const  China  log_market_share  
8           34  SUV       C      CN    257    1.0      1         -4.568514  
9           34  SUV       C      CN    183    1.0      1         -5.119955  
10          34  SUV       C      CN    177    1.0      1         -5.856883  
21          34  SUV       C      CN      3    1.0      1         -9.934421  
28          17  SUV       F      DE    222    1.0      0         -3.203259  


In [10]:
params = OLS_result.params
parms = np.array(params)
print(params)

const        -6.424012
Price        -0.032216
Range         0.083525
HP            0.001661
Chargetime   -0.013187
China        -2.227452
dtype: float64


In [11]:
alpha = params[1]
alpha = np.array(alpha)
beta = params
beta = np.array(beta)
p_j = df2['Price']
p_j = np.array(p_j)
columns = X1.columns
X = np.array(X1)

#prints
print(f'alpha: {alpha.round(5)} \n\n beta {beta.shape}: {beta} \n\n p_j: {p_j.shape} \n\n columns: {columns} \n\n X{X.shape}: {X[0:1, :]}')

alpha: -0.03222 

 beta (6,): [-6.42401187e+00 -3.22156109e-02  8.35247810e-02  1.66053731e-03
 -1.31872621e-02 -2.22745233e+00] 

 p_j: (334,) 

 columns: Index(['const', 'Price', 'Range', 'HP', 'Chargetime', 'China'], dtype='object') 

 X(334, 6): [[ 1.      28.46217 40.      20.1     34.       1.     ]]


In [12]:
df2['logit_market_share'] = fun.ccp(alpha, beta, X, p_j)
ccp = df2['logit_market_share']
ccp = np.array(ccp)

share_j: 334
 choice probability sum: 1.0 
 ccp:[array([[0.00049171]]), array([[0.00044777]]), array([[0.0005246]]), array([[0.00040219]]), array([[0.00051022]]), array([[0.00068096]]), array([[0.0009102]]), array([[0.00094094]]), array([[0.00137031]]), array([[0.00044635]]), array([[0.0004344]])]


In [13]:
probability_ratio = fun.probability_ratio(ccp, df2['Model'])
probability_ratio

Model,U5,U5,U5,U6,e-tron,e-tron,e-tron,e-tron,e-tron,e-tron GT,...,up!,up!,C40,C40,C40,EX30,XC40,XC40,XC40,Free
Model,,,,,,,,,,,,,,,,,,,,,
U5,[[1.0]],[[1.0981412533264618]],[[0.937305538456218]],[[1.222588173385246]],[[0.9637252746319823]],[[0.7220861993926895]],[[0.5402276913730183]],[[0.5225765367928032]],[[0.35883425969423866]],[[1.1016381470920849]],...,[[0.33394834160744336]],[[0.3091978429489364]],[[0.10409507164456802]],[[0.09569978853147403]],[[0.0888568025738175]],[[0.06892476105267664]],[[0.10598966427043802]],[[0.09145098127562941]],[[0.09848549899454571]],[[1.1098667472136328]]
U5,[[0.9106296635071537]],[[1.0]],[[0.8535382270877773]],[[1.113325056937632]],[[0.8775968225514613]],[[0.6575531127761243]],[[0.4919473608122581]],[[0.47587369585636413]],[[0.3267651211602031]],[[1.0031843751931095]],...,[[0.30410326594675813]],[[0.2815647276817277]],[[0.09479206006444603]],[[0.08714706622812196]],[[0.08091564022811702]],[[0.06276493196470989]],[[0.09651733230982515]],[[0.08327797630642542]],[[0.08968381680973728]],[[1.0106775825529295]]
U5,[[1.0668879665930946]],[[1.1715936887934613]],[[1.0]],[[1.3043646102837507]],[[1.0281868986064873]],[[0.7703850769750022]],[[0.5763624231462413]],[[0.5575306187281354]],[[0.3828359536691247]],[[1.175324482672459]],...,[[0.3562854671247013]],[[0.3298794579387617]],[[0.11105777931923566]],[[0.10210095278973348]],[[0.09480025341594421]],[[0.0735349981674051]],[[0.11307909739337238]],[[0.09756795145609942]],[[0.10507299376119712]],[[1.1841034771240446]]
U6,[[0.8179369159371813]],[[0.8982102700092373]],[[0.7666568014157181]],[[1.0]],[[0.7882664789431967]],[[0.590620958972057]],[[0.44187217178550997]],[[0.4274346408454385]],[[0.29350378770690716]],[[0.9010705085112509]],...,[[0.27314867661672854]],[[0.252904330076082]],[[0.0851432018652179]],[[0.07827638988727431]],[[0.0726792590572673]],[[0.05637610648713348]],[[0.08669285911457933]],[[0.07480113358401723]],[[0.0805549253121331]],[[0.9078009843171501]]
e-tron,[[1.0376401100218835]],[[1.139475410921239]],[[0.9725858220478308]],[[1.26860652674292]],[[1.0]],[[0.749265603383114]],[[0.5605619211131667]],[[0.5422463751325393]],[[0.3723408207087509]],[[1.143103928152935]],...,[[0.34651819392717303]],[[0.3208360837760634]],[[0.1080132215940054]],[[0.0993019391008697]],[[0.09220138239888878]],[[0.07151909664193142]],[[0.10997912689475979]],[[0.0948932062724533]],[[0.1021925040122605]],[[1.151642253688384]]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EX30,[[14.508574055639267]],[[15.932463697439493]],[[13.598966817452878]],[[17.73801105310858]],[[13.982279516289406]],[[10.476441098443935]],[[7.837933467192469]],[[7.581840383797884]],[[5.206173430474354]],[[15.983198639602737]],...,[[4.845114244969512]],[[4.4860198022685625]],[[1.5102710557822903]],[[1.3884674690179084]],[[1.2891855004895492]],[[1.0]],[[1.537758893199993]],[[1.326823334298349]],[[1.4288841555689529]],[[16.102583893840457]]
XC40,[[9.434882230105467]],[[10.360833397155583]],[[8.843367368960008]],[[11.53497543180956]],[[9.092634468328802]],[[6.812798251254479]],[[5.096984645546191]],[[4.9304480808564755]],[[3.385558980342223]],[[10.393826178005426]],...,[[3.1507632740052576]],[[2.917245234025861]],[[0.9821247417008904]],[[0.9029162342404553]],[[0.8383534676277007]],[[0.6502970032701643]],[[1.0]],[[0.8628292381631437]],[[0.9291990843867093]],[[10.47146205107086]]
XC40,[[10.934819791446984]],[[12.007976710678593]],[[10.249267152543926]],[[13.368781355122005]],[[10.53816220656348]],[[7.8958824642499135]],[[5.907292451513393]],[[5.714280257067768]],[[3.923787964753788]],[[12.046214613835515]],...,[[3.65166493512997]],[[3.381022692550746]],[[1.1382608496111142]],[[1.0464599416712534]],[[0.9716331233888771]],[[0.7536798412795628]],[[1.1589778785532068]],[[1.0]],[[1.0769211835760906]],[[12.136192873300518]]


In [18]:
marginal_effects = fun.marginal_effects(ccp, df2['Model'], columns[1:], beta)
marginal_effects

,Price,Range,HP,Chargetime,China
Model,,,,,
U5,[[-1.583305488290053e-05]],[[4.10500501015838e-05]],[[8.161067748605848e-07]],[[-6.481163589876712e-06]],[[-0.0010947293561145207]]
U5,[[-1.4418683347165652e-05]],[[3.738303682880884e-05]],[[7.432037121844232e-07]],[[-5.9021993048565875e-06]],[[-0.000996936854789085]]
U5,[[-1.689153987965893e-05]],[[4.379436334183117e-05]],[[8.70665846583108e-07]],[[-6.914447910063731e-06]],[[-0.0011679151441717354]]
U6,[[-1.2951600011288257e-05]],[[3.35793587081698e-05]],[[6.675836465338159e-07]],[[-5.301657768802646e-06]],[[-0.0008954997532613233]]
e-tron,[[-1.6428708590578533e-05]],[[4.259438975834729e-05]],[[8.468094427855121e-07]],[[-6.7249907698448085e-06]],[[-0.0011359140551316258]]
...,...,...,...,...,...
EX30,[[-0.00022818845194663178]],[[0.0005916197129543501]],[[1.1761857895137896e-05]],[[-9.340753867935246e-05]],[[-0.015777409913616283]]
XC40,[[-0.00014876313080718594]],[[0.00038569524441544326]],[[7.66792004443395e-06]],[[-6.0895272203273734e-05]],[[-0.010285782977864387]]
XC40,[[-0.00017228542358241258]],[[0.00044668103042253915]],[[8.880364682319266e-06]],[[-7.052397800974855e-05]],[[-0.011912161754077139]]


In [17]:
elasticity = fun.elasticity(ccp, df2['Model'], columns[1:], beta, X)
elasticity

elasticity shape: 
(334, 5)


,Price,Range,HP,Chargetime,China
Model,,,,,
U5,[[-0.9164753290310935]],[[3.3393484288505726]],[[0.03336038803861721]],[[-0.44814644288635863]],[[-2.2263570606559258]]
U5,[[-0.9165156229696165]],[[3.339495247314787]],[[0.03336185476814298]],[[-0.4481661461829417]],[[-2.2264549451179407]]
U5,[[-0.9164451716016867]],[[3.339238544643854]],[[0.03335929028561133]],[[-0.44813169623220755]],[[-2.22628380041229]]
U6,[[-0.9165574148663319]],[[3.339647523868465]],[[0.033363376024468316]],[[-0.44818658196474753]],[[-2.226556468510206]]
e-tron,[[-0.9164583584305704]],[[3.339286593309126]],[[0.0333597702960542]],[[-0.4481381444477459]],[[-2.226315834651143]]
...,...,...,...,...,...
EX30,[[-0.9103847885464786]],[[3.3171564110690883]],[[0.03313868780567535]],[[-0.44516823499905944]],[[-2.2115615529301818]]
XC40,[[-0.912672337575902]],[[3.325491521699289]],[[0.03322195630277675]],[[-0.4462868215316048]],[[-2.2171186047917284]]
XC40,[[-0.9119960685334318]],[[3.3230274095809187]],[[0.03319733960338435]],[[-0.44595613334369955]],[[-2.215475771307931]]


In [73]:
def print_cross_elasticity(cross_elasticity, model_labels):
    print(cross_elasticity.shape)
    for k in range(cross_elasticity.shape[1]):
        print(f'Change in : {model_labels[k]} \n {cross_elasticity[:,k:k+1,:]}')

def cross_elasticity(ccp, coefficients, X, model_labels):
    cross_elasticity = np.zeros((len(ccp), len(ccp), len(coefficients)-1))
    X = X[:,1:] #remove the constant
    print(coefficients)
    print(X[:1])
    #for k in range(len(coefficients)-1):
    for k in range(len(coefficients)-1):
        for i in range(len(ccp)):
            for j in range(len(ccp)):
                cross_elasticity[i,j, k] = -coefficients[k+1]*X[i,k]*(ccp[j])
                #cross_elasticity[i,j, k] = -coefficients[k+1]*X[:,k]*(ccp[j])
    print_cross_elasticity(cross_elasticity, model_labels)
    #return cross_elasticity

In [74]:
#print(f'beta: {beta} \n\n X:{X[:1]}
Ecross_elasticity = cross_elasticity(ccp, beta, X, np.array(df2['Model']))
#cross_elasticity[0:1, 0:1, :]

[-6.42401187e+00 -3.22156109e-02  8.35247810e-02  1.66053731e-03
 -1.31872621e-02 -2.22745233e+00]
[[28.46217 40.      20.1     34.       1.     ]]
(334, 334, 5)
Change in : U5 
 [[[ 4.50864796e-04 -1.64280980e-03 -1.64118161e-05  2.20467969e-04
    1.09526791e-03]]

 [[ 4.96898493e-04 -1.64280980e-03 -1.64118161e-05  2.20467969e-04
    1.09526791e-03]]

 [[ 4.19028343e-04 -1.64280980e-03 -1.64118161e-05  2.20467969e-04
    1.09526791e-03]]

 ...

 [[ 6.59396360e-04 -1.87691019e-03 -3.28236322e-05  1.81561857e-04
    0.00000000e+00]]

 [[ 6.95835370e-04 -1.87691019e-03 -3.28236322e-05  1.81561857e-04
    0.00000000e+00]]

 [[ 7.99595908e-04 -2.05351224e-03 -3.93556983e-05  3.56639362e-04
    1.09526791e-03]]]
Change in : U5 
 [[[ 4.10570857e-04 -1.49599133e-03 -1.49450866e-05  2.00764673e-04
    9.97383452e-04]]

 [[ 4.52490508e-04 -1.49599133e-03 -1.49450866e-05  2.00764673e-04
    9.97383452e-04]]

 [[ 3.81579639e-04 -1.49599133e-03 -1.49450866e-05  2.00764673e-04
    9.97383452e-04]